In [1]:
import torch
import crypten
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

crypten.init()

In [2]:
iris = sm.datasets.get_rdataset("iris")
y = iris.data['Species']
X = iris.data.drop(columns=['Species'])
X = sm.add_constant(X)
X = X.to_numpy(dtype=np.float64)
#X = MinMaxScaler().fit(X).transform(X)
y = pd.get_dummies(y, drop_first=True)["virginica"].to_numpy(dtype=np.float64)

In [3]:
affairs = sm.datasets.get_rdataset('Affairs', 'AER')
y = affairs.data['affairs'].to_numpy(dtype=np.float64)
X = affairs.data.drop(columns=['affairs'])
X = sm.add_constant(X)
X = pd.get_dummies(X, drop_first=True).to_numpy(dtype=np.float64)
#X = MinMaxScaler().fit(X).transform(X)

In [4]:
np.linalg.inv(X.T @ X);

In [5]:
model = sm.OLS(y,X)
results = model.fit()
params_no_SMPC = results.params

In [6]:
def norm_1(A):
    return A.abs().sum(axis=0).max()

def ge_enc_inv(A):
    n = A.shape[0]
    
    # Create an augmented matrix [A | I]
    augmented_matrix = crypten.cat([A, crypten.cryptensor(np.eye(n)),], dim=1)
    
    # Perform Gaussian elimination
    for i in range(n):
        # Find the pivot element
        pivot = augmented_matrix[i, i]
        if pivot.get_plain_text() == 0:
            # Swap with a non-zero row below
            for j in range(i + 1, n):
                if augmented_matrix[j, i] != 0:
                    augmented_matrix[[i, j]] = augmented_matrix[[j, i]]
                    pivot = augmented_matrix[i, i]
                    break
            else:
                raise ValueError("Matrix is singular and cannot be inverted.")
        
        # Normalize the pivot row
        augmented_matrix[i] = augmented_matrix[i] / pivot
        
        # Eliminate the column entries above and below the pivot
        for j in range(n):
            if j != i:
                factor = augmented_matrix[j, i]
                augmented_matrix[j] = augmented_matrix[j] - factor * augmented_matrix[i]
    
    # The right half of the augmented matrix is now the inverse
    A_inv = augmented_matrix[:, n:]
    return A_inv

def n_enc_inv(A, niter=1000):
    n = A.shape[0]
    
    X_k = crypten.cryptensor(np.eye(n) / norm_1(A).get_plain_text())
    
    for k in range(niter):
        X_k_next = 2 * X_k - X_k.matmul(A).matmul(X_k)       
        X_k = X_k_next
    
    return X_k_next

In [7]:
class SecureOLS:

    def __init__(self, y, X):
        self.y_enc = crypten.cryptensor(y)
        self.X_enc = crypten.cryptensor(X)

    def fit1(self):
        X_enc_t = self.X_enc.t()
        X_X_t_enc = X_enc_t.matmul(self.X_enc)
        n1 = norm_1(X_X_t_enc).get_plain_text()
        return n_enc_inv(X_X_t_enc / n1).matmul(X_enc_t).matmul(self.y_enc) / n1

    def fit2(self):
        X_enc_t = self.X_enc.t()
        X_X_t_enc = X_enc_t.matmul(self.X_enc)
        n1 = norm_1(X_X_t_enc).get_plain_text()
        return ge_enc_inv(X_X_t_enc / n1).matmul(X_enc_t).matmul(self.y_enc) / n1

In [8]:
model = SecureOLS(y, X)
X_enc1 = model.fit1()
X_enc2 = model.fit2()

In [9]:
params_SMPC1 = np.array(X_enc1.get_plain_text())
params_SMPC2 = np.array(X_enc2.get_plain_text())

In [10]:
# Crypten results are totally different from the true values
params_SMPC1

array([379.15674  ,  -4.438019 ,   3.7505798,  -5.2733154, -19.051392 ,
         2.0276337,  -2.3361664, 101.151825 ,  25.98584  ], dtype=float32)

In [11]:
params_SMPC2

array([ 1.4877777 , -0.02581787,  0.13748169, -0.34294128,  0.13941956,
        0.07505798, -0.4957428 ,  0.01974487,  0.1219635 ], dtype=float32)

In [12]:
params_no_SMPC

array([ 5.87201014, -0.05097628,  0.16947232, -0.47761363, -0.01374903,
        0.10491597, -0.71187692,  0.05408587, -0.14262446])

In [13]:
f"avg. error: {100 * sum(np.abs((params_SMPC2 - params_no_SMPC)/ params_no_SMPC)) / len(params_no_SMPC)} %"

'avg. error: 176.99436705570864 %'